In [1]:
import os
import nltk
import nltk.corpus
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import string
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist
from nltk.util import bigrams, trigrams, ngrams
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [2]:
df = pd.read_csv ("train.csv")
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         60000 non-null  int64 
 1   comment    60000 non-null  object
 2   subreddit  60000 non-null  object
dtypes: int64(1), object(2)
memory usage: 937.6+ KB


(60000, 3)

In [3]:
def clean_text(df):
    
    all_comments = list()
    lines = df["comment"].values.tolist()
    for text in lines:
        text = text.lower()
        
        pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        text = pattern.sub('', text)
        
        emoji = re.compile("["
                           u"\U0001F600-\U0001FFFF"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
        text = emoji.sub(r'', text)
        
        text = re.sub(r"i'm", "i am", text)
        text = re.sub(r"he's", "he is", text)
        text = re.sub(r"she's", "she is", text)
        text = re.sub(r"that's", "that is", text)        
        text = re.sub(r"what's", "what is", text)
        text = re.sub(r"where's", "where is", text) 
        text = re.sub(r"\'ll", " will", text)  
        text = re.sub(r"\'ve", " have", text)  
        text = re.sub(r"\'re", " are", text)
        text = re.sub(r"\'d", " would", text)
        text = re.sub(r"\'ve", " have", text)
        text = re.sub(r"won't", "will not", text)
        text = re.sub(r"don't", "do not", text)
        text = re.sub(r"did't", "did not", text)
        text = re.sub(r"can't", "can not", text)
        text = re.sub(r"it's", "it is", text)
        text = re.sub(r"couldn't", "could not", text)
        text = re.sub(r"have't", "have not", text)
        text = re.sub(r"nt", "not", text)
        
        text = re.sub(r"[,.\"!@#$%^&*(){}?/;`~:<>+=-]", '', text)
        
        tokens = word_tokenize(text)
        
        table = str.maketrans('', '', string.punctuation)
        
        stripped = [w.translate(table) for w in tokens]
        words = [word for word in stripped if word.isalpha()]
        
        stop_words = set(stopwords.words("english"))
        stop_words.discard("not")
        
        words = [w for w in words if not w in stop_words]
        words = ' '.join(words)
        
        all_comments.append(words)
    return all_comments

all_comments = clean_text(df)
all_comments[0:10]

['think prestige poinots not expire ever skins buy available set duration exemple year release another skin vault old one making also limitededition skin also please love god not rerelease skins need grind prestige shop would suck everyone grinded',
 'whats going happen refused asilum appeal',
 'anecdotal evidence anecdotal clearly everyone meanot like people not',
 'look dude due respect music people looks like carti either caught much flak maybe senot polite post inviting discussion instead capitalizing every impactful word post carti',
 'hope gets doomhammer back',
 'trading coaches happened',
 'considering kid already seen nt figure would matter',
 'nah clearly tom bombadil',
 'time go play elite dangerous vr think thanks',
 '']

In [4]:
len(all_comments)

60000

In [5]:
# c = all_comments
# filtered_sentence = [] 
# freq_count_limit = FreqDist()
# lemmatizer=WordNetLemmatizer()
# stop_words = set(stopwords.words('english'))

# for i in c:
#     comment_tokens = word_tokenize(i)
#     for words in comment_tokens:
#         if words not in stop_words: 
#             filtered_sentence.append(words) 
        
#             limit_words = lemmatizer.lemmatize(words)
# #     for word in root_words:
#             freq_count_limit[limit_words.lower()]+=1
# freq_count_limit

FreqDist({'like': 8954, 'would': 7132, 'nt': 6837, 'one': 5971, 'people': 5884, 'get': 5839, 'think': 4498, 'time': 4385, 'game': 3900, 'even': 3839, ...})

In [30]:
# freq_count_limit.plot(20,cumulative=False)
# plt.show()

In [7]:
# freq_count_limit.pop("nt")
# freq_count_limit.pop("would")
# freq_count_limit.pop("one")
# freq_count_limit.pop("get")

In [29]:
# freq_count_limit.plot(20,cumulative=False)
# plt.show()

In [9]:
# lists = top_100_words
# x, y = zip(*lists)
# x=list(x)

In [10]:
# text_counts.shape

In [11]:
token = RegexpTokenizer(r'[a-z]+')
cv = CountVectorizer(ngram_range = (1,1), tokenizer = token.tokenize)
text_counts2 = cv.fit_transform(all_comments)

In [12]:
text_counts2.shape

(60000, 57416)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    text_counts2, df['subreddit'], test_size=0.2, random_state=1)

In [14]:
clf = MultinomialNB().fit(X_train, y_train)
predicted= clf.predict(X_test)
print("MultinomialNB Accuracy:",metrics.accuracy_score(y_test, predicted))

MultinomialNB Accuracy: 0.46875


# TEST

In [15]:
df_test = pd.read_csv ("test.csv")

In [16]:
df_test.head

<bound method NDFrame.head of           id                                            comment
0          0                          Holy shit a shot counter.
1          1  It doesn't matter that it isn't hard to rememb...
2          2             I find it funny that this is downvoted
3          3  They are really getting ridicoulous with all t...
4          4                            He's Eden's best friend
...      ...                                                ...
19995  19995  These officials are almost as incompetent as o...
19996  19996  honestly the Patriot act really fucked our com...
19997  19997  My friend is now looking online for a thanos c...
19998  19998  I really liked Thor Ragnarok and both Guardian...
19999  19999                      last info changes everything.

[20000 rows x 2 columns]>

In [17]:
cleaned_test = clean_text(df_test)

In [18]:
# d = cleaned_test
# filtered_sentence_test = [] 
# freq_count_limit_test = FreqDist()
# lemmatizer_test=WordNetLemmatizer()
# stop_words_test = set(stopwords.words('english'))

# for i in d:
#     comment_tokens_test = word_tokenize(i)
#     for words in comment_tokens_test:
#         if words not in stop_words_test: 
#             filtered_sentence_test.append(words) 
        
#             limit_words_test = lemmatizer.lemmatize(words)
# #     for word in root_words:
#             freq_count_limit_test[limit_words_test.lower()]+=1
# freq_count_limit_test

FreqDist({'like': 2896, 'would': 2317, 'nt': 2229, 'one': 2023, 'people': 1981, 'get': 1920, 'time': 1515, 'think': 1376, 'game': 1303, 'really': 1215, ...})

In [19]:
token_test = RegexpTokenizer(r'[a-z]+')
cv_test = CountVectorizer(ngram_range = (1,1), tokenizer = token_test.tokenize)
text_counts_test = cv.fit_transform(cleaned_test)

In [20]:
# X_train, X_test, y_train, y_test = train_test_split(
#     text_counts_test, df['subreddit'], test_size=0.2, random_state=1)

In [21]:
#########################################################################

In [22]:
############ Second Try

In [23]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(df.comment,df., test_size=0.2)

In [24]:
# from sklearn.feature_extraction.text import CountVectorizer
# v = CountVectorizer()
# X_train_count = v.fit_transform(X_train.values)
# X_train_count.toarray()[:20]

In [25]:
# from sklearn.naive_bayes import MultinomialNB
# model = MultinomialNB()
# model.fit(X_train,y_train)
# from collections import defaultdict

In [26]:
# # noinspection SpellCheckingInspection
# class NaiveBayesClassifier(object):
#     def __init__(self, n_gram=1, printing=False):
#         self.prior = defaultdict(int)
#         self.logprior = {}
#         self.bigdoc = defaultdict(list)
#         self.loglikelihoods = defaultdict(defaultdict)
#         self.V = []
#         self.n = n_gram

#         training_set = all_comments
#         training_labels = df['subreddit']
        
#     def compute_prior_and_bigdoc(self, training_set, training_labels):
        
#         for x, y in zip(training_set, training_labels):
#             all_words = x.split(" ")
#             if self.n == 1:
#                 grams = all_words
#             else:
#                 grams = self.words_to_grams(all_words)

#             self.prior[y] += len(grams)
#             self.bigdoc[y].append(x)

#     def compute_vocabulary(self, documents):
#         vocabulary = set()

#         for doc in documents:
#             for word in doc.split(" "):
#                 vocabulary.add(word.lower())

#         return vocabulary

#     def count_word_in_classes(self):
#         counts = {}
#         for c in list(self.bigdoc.keys()):
#             docs = self.bigdoc[c]
#             counts[c] = defaultdict(int)
#             for doc in docs:
#                 words = doc.split(" ")
#                 for word in words:
#                     counts[c][word] += 1

#         return counts

#     def train(self, training_set, training_labels, alpha=1):
#         # Get number of documents
#         N_doc = len(training_set)

#         # Get vocabulary used in training set
#         self.V = self.compute_vocabulary(training_set)

#         # Create bigdoc
#         for x, y in zip(training_set, training_labels):
#             self.bigdoc[y].append(x)

#         # Get set of all classes
#         all_classes = set(training_labels)

#         # Compute a dictionary with all word counts for each class
#         self.word_count = self.count_word_in_classes()

#         # For each class
#         for c in all_classes:
#             # Get number of documents for that class
#             N_c = float(sum(training_labels == c))

#             # Compute logprior for class
#             self.logprior[c] = np.log(N_c / N_doc)

#             # Calculate the sum of counts of words in current class
#             total_count = 0
#             for word in self.V:
#                 total_count += self.word_count[c][word]

#             # For every word, get the count and compute the log-likelihood for this class
#             for word in self.V:
#                 count = self.word_count[c][word]
#                 self.loglikelihoods[c][word] = np.log((count + alpha) / (total_count + alpha * len(self.V)))

#     def predict(self, test_doc):
#         sums = {
#             0: 0,
#             1: 0,
#         }
#         for c in self.bigdoc.keys():
#             sums[c] = self.logprior[c]
#             words = test_doc.split(" ")
#             for word in words:
#                if word in self.V:
#                    sums[c] += self.loglikelihoods[c][word]

#         return sums

In [27]:
# NBclassifier = NaiveBayesClassifier(n_gram=1)
# NBclassifier.train(X_train, y_train)

In [28]:
# result = NBclassifier.predict(test)
# print(np.exp(result))